# PS1: Let's build a backtester for U.S. Treasury Portfolios
In this problem set, we will develop an initial proof-of-concept backtesting system for understanding how the value of United States Treasury security portfolio's change as a function of market conditions and time.

Fill me in

Let's go!
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 

> The [include(...) function](https://docs.julialang.org/en/v1/base/base/#include) is a convenient way to load Julia code from another file. The `Include.jl` file is a Julia file that loads external packages, and various functions that we will use in the exercise. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Otherwise, packages are downloaded and then loaded.

In [1]:
include(joinpath(@__DIR__, "Include-solution.jl")); # this sets up the environment, we'll do this all the time, on everything we do

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Constants
Let's define some constants that we will use throughout the problem set. See the comment next to each constant for a description of its purpose, permissible values, units, etc.

In [2]:
Δt = (1/252); # units: years, time step for the lattice model, daily steps (trading days basis)
number_of_steps = 10; # project value out to this many time steps
μ₊ = 1.21; # up factor drift
μ₋ = -(1/μ₊); # down factor drift
μₒ = 0.0; # neutral factor drift

## Task 1: Recombining lattice model for the investment rate of a U.S. Treasury Bill
Fill me in

In [3]:
interest_rate_tree_4_week = let
    
    # initialize -
    Δ = [
        exp(μ₊*Δt) , exp(μₒ*Δt), exp(μ₋*Δt) # this is the amount by which the interest rate changes at each step (daily)
    ];
    r̄ = 0.0425; # initial interest rate (4.25% annualized)
    n = 3; # number of branches at each node
    h = number_of_steps - 1; # number of time steps to project out

    # build an empty tree -
    interest_rate_tree = build(MyGeneralDiscreteRecombiningTreeModel, (
        n = n, h = h,
    )) |> tree -> populate!(tree, r̄, Δ);


    interest_rate_tree # return - interest_rate_tree
end

MyGeneralDiscreteRecombiningTreeModel(Dict{Int64, NamedTuple}(56 => (quantity = 0.04374221262160997, path = [6, 0, 0]), 35 => (quantity = 0.04353268400647249, path = [5, 0, 0]), 60 => (quantity = 0.04291010090427576, path = [2, 4, 0]), 67 => (quantity = 0.042564735891883254, path = [1, 4, 1]), 215 => (quantity = 0.041735363796841236, path = [1, 1, 7]), 73 => (quantity = 0.042222150574464004, path = [0, 4, 2]), 115 => (quantity = 0.04201000984631323, path = [1, 1, 5]), 112 => (quantity = 0.04214800988616441, path = [1, 2, 4]), 185 => (quantity = 0.0434562420651726, path = [6, 1, 2]), 86 => (quantity = 0.04353268400647249, path = [5, 2, 0])…), Dict(56 => [84, 85, 92], 35 => [56, 57, 63], 60 => [88, 89, 96], 67 => [96, 97, 103], 215 => [277, 278, 281], 73 => [103, 104, 109], 115 => [156, 157, 160], 112 => [152, 153, 157], 185 => [242, 243, 251], 86 => [122, 123, 131]…), Dict(0 => [0], 4 => [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], 5 => [35, 36, 37, 38, 39, 40, 41, 42, 

Fill me in

In [4]:
interest_rate_tree_4_week |> typeof |> fieldnames

(:data, :connectivity, :levels, :h, :n)

In [9]:
interest_rate_tree_4_week.levels[2]

6-element Vector{Int64}:
 4
 5
 6
 7
 8
 9

In [10]:
nodes_at_level(2,3)

6

## Task 2: Compute the captial gains and losses of a U.S. Treasury Bill portfolio
Fill me in

## Task 3: Compute the Net Present Value of a U.S. Treasury Bill portfolio
Fill me in

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___